In [1]:
import autograd.numpy as np
from autograd import grad

In [2]:
class ORNeuron():
    def __init__(self, n):
        self.n = n
        self.weights = np.random.rand(n)
        
    def getWeights(self):
        return self.weights    
    
    def present(self, inputs):
        z = np.sum(np.multiply(inputs, self.weights))
        return 1 - np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [3]:
class ANDNeuron():
    def __init__(self, n):
        self.weights = np.random.rand(n)
        
    def getWeights(self):
        return self.weights
        
    def present(self, inputs):
        i = 1.0 - inputs
        i[0] = 1.0
        
        z = np.sum(np.multiply(i, self.weights))
        return np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [4]:
class CNFNetwork():
    def __init__(self, n, c):
        self.n = n
        self.c = c
        
        d = []
        for i in range(0, c):
            d.append(ORNeuron(n*2 + 1))
        
        self.disjunctions = np.array(d)
        
        if not c == 0:
            self.conjunction = ANDNeuron(c + 1)
        else:
            self.conjunction = ANDNeuron(n*2 + 1)
        
    def getDisjunctions(self):
        return self.disjunctions
    
    def getConjunction(self):
        return self.conjunction
    
    def fowardprop(self, inputs):
        actualIn = self.__convertInputs__(inputs)
        
        if not self.c == 0:
            dout = [1]
            for d in self.disjunctions:
                dout.append(d.present(actualIn))
            
            actualIn = dout
            
        return self.conjunction.present(np.array(actualIn))
    
    def __convertInputs__(self, inputs):
        actual = [1]
        
        for i in inputs:
            actual.append(i)
            actual.append(1-i)
        
        return np.array(actual)
    
    def __repr__(self):
        s = "Disjunctions -> "
        for d in self.disjunctions:
            s += (str(d) + ", ")
            
        s += ("\nConjunction -> " + str(self.conjunction) + "\n")
            
        return s

In [5]:
def SSE(network, data, targets, p=False):
    predictions = predict(network, data)
    
#     if p:
#         print(predictions)
    
    return np.sum(np.power(np.subtract(targets, predictions), 2.0))
    
def predict(network, data):
    t = np.array([network.fowardprop(d) for d in data])
    return t
    

In [6]:
def trainCNFNetwork(data, targets, inputNodes, numC, it=10000, lr=0.1):
    network = CNFNetwork(inputNodes, numC)
    print(network)
    print("Initial Loss: ", SSE(network, data, targets, True))
    
    pterb = 0.0001
    
    for i in range(1, it):
        if i%1000 == 0:
            print("Iteration -> " + str(i) + " : " + str(SSE(network, data, targets)))
            
        for d in network.getDisjunctions():
            g = computeNeuronGrad(network, d, pterb, data, targets)
            d.setGrad(g * lr)
            
        g = computeNeuronGrad(network, network.getConjunction(), pterb, data, targets)
        
        network.getConjunction().updateWeights(g * lr)
        for d in network.getDisjunctions():
            d.applyGrad()
            
    print("Trained Loss: ", SSE(network, data, targets, True))
    return network

def computeNeuronGrad(network, neuron, pterb, data, targets):
    gradient = np.zeros(len(neuron.getWeights()))
    for k in range(0, len(neuron.getWeights())):
        g = np.zeros(len(neuron.getWeights()))
        g[k] = -pterb

        oldSSE = SSE(network, data, targets)
        neuron.updateWeights(g)
        newSSE = SSE(network, data, targets)
        neuron.updateWeights(-g)
                
        gradient[k] = (newSSE - oldSSE)/pterb
        
    return gradient

# NOT

In [7]:
dataNOT = np.array([[1.0], [0.0]])
targetsNOT = np.array([0.0, 1.0])
trainCNFNetwork(dataNOT, targetsNOT, 1, 0)

Disjunctions -> 
Conjunction -> [ 0.08384683  0.95554502]

Initial Loss:  0.4354419721
Iteration -> 1000 : 0.00253492636954
Iteration -> 2000 : 0.00126068258724
Iteration -> 3000 : 0.000838586930959
Iteration -> 4000 : 0.000628157334929
Iteration -> 5000 : 0.000502121398909
Iteration -> 6000 : 0.000418197256062
Iteration -> 7000 : 0.00035830321831
Iteration -> 8000 : 0.000313412325618
Iteration -> 9000 : 0.000278515420617
Trained Loss:  0.000250609755358


Disjunctions -> 
Conjunction -> [ 0.          4.14568644]

# AND
For this we dont neeed any disjunctions

In [8]:
dataAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsAND = np.array([0.0, 0.0, 0.0, 1.0])
trainCNFNetwork(dataAND, targetsAND, 2, 0)

Disjunctions -> 
Conjunction -> [ 0.09729661  0.40437016  0.59076866  0.74971913]

Initial Loss:  1.02939324302
Iteration -> 1000 : 0.00513952254646
Iteration -> 2000 : 0.00254075061141
Iteration -> 3000 : 0.00168631145204
Iteration -> 4000 : 0.00126166488227
Iteration -> 5000 : 0.00100777184534
Iteration -> 6000 : 0.000838904869129
Iteration -> 7000 : 0.00071848780167
Iteration -> 8000 : 0.000628289567814
Iteration -> 9000 : 0.000558205313526
Trained Loss:  0.000502183092491


Disjunctions -> 
Conjunction -> [ 4.14414572  0.          4.14517597  0.        ]

# OR

In [9]:
dataOR = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsOR = np.array([0.0, 1.0, 1.0, 1.0])
trainCNFNetwork(dataOR, targetsOR, 2, 1)

Disjunctions -> [ 0.03597072  0.54622006  0.28498513  0.46150847], 
Conjunction -> [ 0.91897467]

Initial Loss:  0.859634250643
Iteration -> 1000 : 0.0147264514446
Iteration -> 2000 : 0.00665424999811
Iteration -> 3000 : 0.00422199561736
Iteration -> 4000 : 0.00306961003498
Iteration -> 5000 : 0.00240224827008
Iteration -> 6000 : 0.00196868011984
Iteration -> 7000 : 0.00166512071001
Iteration -> 8000 : 0.0014410967145
Iteration -> 9000 : 0.0012691786224
Trained Loss:  0.00113320461181


Disjunctions -> [ 5.33022     0.          5.33026365  0.        ], 
Conjunction -> [ 3.85379597]

# NAND

In [10]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 1)

Disjunctions -> [ 0.2670529   0.71459223  0.52982119  0.41614743], 
Conjunction -> [ 0.24131879]

Initial Loss:  0.736507950989
Iteration -> 1000 : 0.0145598993122
Iteration -> 2000 : 0.00661740881009
Iteration -> 3000 : 0.00420662032181
Iteration -> 4000 : 0.00306130179544
Iteration -> 5000 : 0.00239708104691
Iteration -> 6000 : 0.00196516924452
Iteration -> 7000 : 0.00166258581079
Iteration -> 8000 : 0.00143918357024
Iteration -> 9000 : 0.00126768519266
Trained Loss:  0.00113200746639


Disjunctions -> [ 0.          5.33094093  0.          5.33087605], 
Conjunction -> [ 3.85433414]

# NOR

In [11]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 0.0, 0.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 2)

Disjunctions -> [ 0.6713813   0.45593178  0.11943221  0.55086449], [ 0.04325333  0.15126712  0.38242037  0.26897606], 
Conjunction -> [ 0.04322681  0.54394268]

Initial Loss:  0.820470053067
Iteration -> 1000 : 0.0241405118157
Iteration -> 2000 : 0.00861527176654
Iteration -> 3000 : 0.00529972730603
Iteration -> 4000 : 0.00382827656174
Iteration -> 5000 : 0.00299095435523
Iteration -> 6000 : 0.00245027690704
Iteration -> 7000 : 0.00207263608272
Iteration -> 8000 : 0.00179421311504
Iteration -> 9000 : 0.00158062496467
Trained Loss:  0.00141170429587


Disjunctions -> [ 0.          0.          0.          5.66369549], [ 0.          5.70631119  0.          0.        ], 
Conjunction -> [ 3.94126039  3.98088195]

# XOR

In [12]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([0.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataNAND, targetsNAND, 2, 2)

Disjunctions -> [ 0.23628204  0.98037393  0.77622604  0.38387031], [ 0.00459108  0.37895811  0.05353995  0.40983986], 
Conjunction -> [ 0.21337505  0.6377396 ]

Initial Loss:  1.03906513532
Iteration -> 1000 : 0.716665358151
Iteration -> 2000 : 0.686879187644
Iteration -> 3000 : 0.679023421963
Iteration -> 4000 : 0.675493209024
Iteration -> 5000 : 0.673505165411
Iteration -> 6000 : 0.672235507916
Iteration -> 7000 : 0.671356701026
Iteration -> 8000 : 0.670713415296
Iteration -> 9000 : 0.670222729861
Trained Loss:  0.66983643155


Disjunctions -> [ 0.30954545  0.93013653  0.75667035  0.42645202], [ 0.          7.05349     0.          7.05349383], 
Conjunction -> [ 0.          3.24415127]

# IMPLIES

In [13]:
dataIMPLYS = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsIMPLYS = np.array([1.0, 1.0, 0.0, 1.0])
trainCNFNetwork(dataIMPLYS, targetsIMPLYS, 2, 1)

Disjunctions -> [ 0.66001029  0.21698503  0.21025963  0.27813663], 
Conjunction -> [ 0.64862298]

Initial Loss:  0.852875890091
Iteration -> 1000 : 0.0148887839343
Iteration -> 2000 : 0.00668978603467
Iteration -> 3000 : 0.00423677449114
Iteration -> 4000 : 0.00307758211592
Iteration -> 5000 : 0.00240720127841
Iteration -> 6000 : 0.00197204312843
Iteration -> 7000 : 0.00166754765698
Iteration -> 8000 : 0.00144292771334
Iteration -> 9000 : 0.0012706075217
Trained Loss:  0.00113434976913


Disjunctions -> [ 0.          5.32961667  5.32959321  0.        ], 
Conjunction -> [ 3.85328125]

# Complicated Formula

## (a AND b) OR (NOT c)
CNF: (a OR (NOT c)) AND (b OR (NOT c))

In [14]:
dataInteresting1 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting1 = np.array([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0])
trainCNFNetwork(dataInteresting1, targetsInteresting1, 3, 2)

Disjunctions -> [ 0.6813627   0.2349668   0.38974434  0.10181127  0.58089235  0.11291412], [ 0.05060651  0.85777213  0.33928586  0.26204975  0.71943256  0.19869621], 
Conjunction -> [ 0.91408946  0.76246137]

Initial Loss:  2.48640800614
Iteration -> 1000 : 0.205994548385
Iteration -> 2000 : 0.162132142219
Iteration -> 3000 : 0.140825971465
Iteration -> 4000 : 0.127388962218
Iteration -> 5000 : 0.117833087967
Iteration -> 6000 : 0.110544903219
Iteration -> 7000 : 0.104724836785
Iteration -> 8000 : 0.0999233713425
Iteration -> 9000 : 0.0958648665695
Iteration -> 10000 : 0.0923691898483
Iteration -> 11000 : 0.0893127360148
Iteration -> 12000 : 0.0866073464618


KeyboardInterrupt: 

## NOT ( a AND b AND c)
CNF: (NOT a) OR (NOT b) OR (NOT c)

In [ ]:
dataInteresting2 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting2 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0])
trainCNFNetwork(dataInteresting2, targetsInteresting2, 3, 1)

## a AND b AND c

In [ ]:
dataInteresting3 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting3 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
trainCNFNetwork(dataInteresting3, targetsInteresting3, 3, 3)

# (a XOR b) AND c
CNF: ((NOT a) OR (NOT b)) AND (a OR b) AND c

In [16]:
dataInteresting4 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting4 = np.array([0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0])
trainCNFNetwork(dataInteresting4, targetsInteresting4, 3, 3)

Disjunctions -> [ 0.98084238  0.96278776  0.38397388  0.92955681  0.57566421  0.77579778], [ 0.13866104  0.72219147  0.22187293  0.64347105  0.6125109   0.2770585 ], [ 0.48088155  0.46536164  0.72479089  0.71587101  0.09710715  0.92288741], 
Conjunction -> [ 0.34922507  0.14265497  0.67395269]

Initial Loss:  2.09120005418
Iteration -> 1000 : 0.232241187669
Iteration -> 2000 : 0.0292496595654
Iteration -> 3000 : 0.0145196644042
Iteration -> 4000 : 0.00951246378153
Iteration -> 5000 : 0.00702631019388
Iteration -> 6000 : 0.00554994713433
Iteration -> 7000 : 0.00457570344681
Iteration -> 8000 : 0.00388632729602
Iteration -> 9000 : 0.00337367697701
Trained Loss:  0.00297800261913


Disjunctions -> [ 0.          5.66913734  0.          5.66777796  0.          0.27527122], [ 0.          0.09039051  0.          0.09040301  6.60066505  0.        ], [ 5.80701829  0.          5.80574025  0.          0.          0.        ], 
Conjunction -> [ 3.78002502  5.36151565  3.82803829]